In [ ]:
# Import Keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, LSTM, RepeatVector, BatchNormalization, Embedding, Input, Concatenate, Bidirectional, TimeDistributed
from keras_applications.inception_v3 import InceptionV3
from keras.models import Model
from keras_preprocessing.image import load_img, img_to_array
from keras_preprocessing.text import Tokenizer, one_hot
from keras.applications import inception_v3


import cv2
import numpy as np

import json
import os

Using TensorFlow backend.


In [ ]:
# def read_json_file(filepath):
#     with open(filepath, 'r') as f:
#         datastore = json.load(f)
#     return datastore
# 
# 
# def generate_dataset(datastore, image_folder_path):
#     X = []
#     y = []
#     count = 0 # in this laptop, I just load 1000 images due to limited amount of RAM size
#     for filename in os.listdir(image_folder_path):
#         for data in datastore["images"]:
#             if data["filename"] == filename:
# #                 img = cv2.resize(cv2.imread(image_folder_path+"/"+filename), (224, 224))
#                 img = cv2.resize(cv2.imread(image_folder_path+"/"+filename), (224, 224))
#                 for sentence in data["sentences"]:
#                     X.append(img)
#                     y.append(sentence["tokens"])
#                 break
#         if count >= 1:
#             break
#         count += 1
#     return np.array(X), np.array(y)


def load_dataset(file_path, img_size=(299, 299)):
    
    X = []
    y = []
    count = 0
    with open(file_path+"/Flickr8k_text/Flickr8k.token.txt") as f:
        "datasets/flickr8k/Flickr8k_text/Flickr8k.token.txt"
        "datasets/flickr8k/Flickr8k_Dataset/Flicker8k_Dataset/667626_18933d713e.jpg"
        for line in f:
            splitted_line = line.split("#")
            file_name = splitted_line[0]
            y.append(splitted_line[-1][2:].split("\n")[0].split(" "))
            X.append(img_to_array(load_img(file_path+"/Flicker8k_Dataset/"+file_name, target_size=img_size)))
            count += 1
            if count >= 2:
                break
    return X, y


def tokens_to_one_hot(tokens):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(tokens)
    # unique_word = len(tokenizer.word_counts)
    one_hot_result = []
    
    for token in tokens:
        one_hot_result.append(tokenizer.texts_to_matrix(token))
        # one_hot_result.append(to_categorical(one_hot(token, unique_word), unique_word))
    return np.array(one_hot_result)

X, y = load_dataset("datasets/flickr8k/")
one_hot_y = tokens_to_one_hot(y)

print(y[0], one_hot_y[0])

['A', 'child', 'in', 'a', 'pink', 'dress', 'is', 'climbing', 'up', 'a', 'set', 'of', 'stairs', 'in', 'an', 'entry', 'way', '.'] [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 

In [3]:
import tensorflow as tf

def categorical_accuracy_with_variable_timestep(y_true, y_pred):
  y_true = y_true[:, :-1, :]  # Discard the last timestep
  y_pred = y_pred[:, :-1, :]  # Discard the last timestep

  # Flatten the timestep dimension
  shape = tf.shape(y_true)
  y_true = tf.reshape(y_true, [-1, shape[-1]])
  y_pred = tf.reshape(y_pred, [-1, shape[-1]])

  # Discard rows that are all zeros as they represent padding words.
  is_zero_y_true = tf.equal(y_true, 0)
  is_zero_row_y_true = tf.reduce_all(is_zero_y_true, axis=-1)
  y_true = tf.boolean_mask(y_true, ~is_zero_row_y_true)
  y_pred = tf.boolean_mask(y_pred, ~is_zero_row_y_true)

  accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_true, axis=1),
                                              tf.argmax(y_pred, axis=1)),
                                    dtype=tf.float32))
  return accuracy


def categorical_crossentropy_from_logits(y_true, y_pred):
  y_true = y_true[:, :-1, :]  # Discard the last timestep
  y_pred = y_pred[:, :-1, :]  # Discard the last timestep
  loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true,
                                                 logits=y_pred)
  return loss

In [71]:
# define the parameter
embedding_size = 300
vocab_size = 2536
lstm_output_size = 300
lstm_layers = 3
dropout_rate = 0.22


#build encoder and decoder model
def image_encoder():
    cnn_model = InceptionV3(include_top=False, weights="imagenet", pooling="max", input_shape=(299, 299, 3))
    
    for layer in cnn_model.layers:
        layer.trainable = False
    
    dense_input = BatchNormalization(axis=-1)(cnn_model.output)
    image_dense = Dense(units=embedding_size)(dense_input)
    
    image_embedding = RepeatVector(1)(image_dense)
    image_input = cnn_model.input
    
    return image_input, image_embedding


def word_embedder():
    sentence_input = Input(shape=[None])
    word_embedder = Embedding(input_dim=vocab_size,
                             output_dim=embedding_size)(sentence_input)    
    return sentence_input, word_embedder


def concantenate_input(image_embedding, word_embedding):
    Concatenate(axis=1)([image_embedding, word_embedding])


def sequence_model(sequnce_input):
    def lstm():
        lstm = LSTM(units=lstm_output_size, return_sequences=True, implementation=2)
        lstm = Bidirectional(lstm)
        return lstm
    
    input_lstm = sequnce_input
    
    for _ in range(lstm_layers):
        input_lstm = BatchNormalization(axis=-1)(input_lstm)
        rnn_out = lstm()(input_lstm)
        input_lstm = rnn_out
    
    time_dist_dense = TimeDistributed(Dense(units=vocab_size))(rnn_out)
    
    return time_dist_dense
    
    
def build():
    image_input, image_embedding = image_encoder()
    sentence_input, word_embedding = word_embedder()
    sequnce_input = Concatenate(axis=1)([image_embedding, word_embedding])
    sequence_output = sequence_model(sequnce_input)
    
    model = Model(inputs=[image_input, sentence_input],
                 outputs=sequence_output)
    model.compile(optimizer="adam", loss=categorical_crossentropy_from_logits, metrics=[categorical_accuracy_with_variable_timestep])
    return model
    
    
print(y)

[['A', 'child', 'in', 'a', 'pink', 'dress', 'is', 'climbing', 'up', 'a', 'set', 'of', 'stairs', 'in', 'an', 'entry', 'way', '.'], ['A', 'girl', 'going', 'into', 'a', 'wooden', 'building', '.']]


In [75]:
model = build()
model.fit([X, one_hot_y], one_hot_y, batch_size=32, epochs=5)

ValueError: setting an array element with a sequence.

In [74]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_753 (Conv2D)             (None, 149, 149, 32) 864         input_17[0][0]                   
__________________________________________________________________________________________________
batch_normalization_785 (BatchN (None, 149, 149, 32) 96          conv2d_753[0][0]                 
__________________________________________________________________________________________________
activation_753 (Activation)     (None, 149, 149, 32) 0           batch_normalization_785[0][0]    
__________________________________________________________________________________________________
conv2d_754